In [1]:
import jax
import jax.nn as jnn
import jax.numpy as jnp
import numpy as np

import equinox as eqx
import diffrax
import optax
from functools import partial

In [2]:
import os
os.chdir("..")
from policy.policy_training import DPCTrainer
from exciting_environments.pmsm.pmsm_env import PMSM, step_eps
import jax_dataclasses as jdc
from models.models import NeuralEulerODE,MLP
#from policy.networks import MLP#,MLP2
import matplotlib.pyplot as plt

2025-02-19 10:15:54.449759: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.6.77). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [3]:
gpus = jax.devices()
jax.config.update("jax_default_device", gpus[1])
jax.config.update("jax_enable_x64", False)

In [4]:
class ExpertModel(eqx.Module):
    motor_env: PMSM = eqx.field(static=True)
    psi_dq_mlp: MLP

    def __init__(self, motor_env, psi_layer_sizes, key):
        self.motor_env = motor_env
        key, subkey = jax.random.split(key)
        self.psi_dq_mlp = MLP(
            psi_layer_sizes, key=subkey, hidden_activation=jax.nn.swish, output_activation=jax.nn.tanh
        )

    def __call__(self, init_obs, actions, tau):

        def body_fun(carry, action):
            obs = carry
            obs = self.step(obs, action, tau)
            return obs, obs

        _, observations = jax.lax.scan(body_fun, init_obs, actions)
        observations = jnp.concatenate([init_obs[None, :], observations], axis=0)
        return observations

    def step(self, obs, action, tau):
        obs1, _ = self.motor_env.reset(self.motor_env.env_properties)  #
        obs1 = obs1.at[2].set((3 * 1500 / 60 * 2 * jnp.pi) / (2 * jnp.pi * 3 * 11000 / 60))
        obs1 = obs1.at[0].set(obs[0])
        obs1 = obs1.at[1].set(obs[1])
        obs1 = obs1.at[4].set(obs[2])
        obs1 = obs1.at[5].set(obs[3])
        state = self.motor_env.generate_state_from_observation(obs1, self.motor_env.env_properties)
        # obs,_= self.motor_env.step(state, action, self.motor_env.env_properties)
        obs, _ = self.step_expert(state, action, self.motor_env.env_properties)
        return jnp.concatenate([obs[0:2], obs[4:6]])

    #@partial(jax.jit, static_argnums=[0, 3])
    def ode_step(self, state, u_dq, properties):
        """Computes state by simulating one step.

        Args:
            system_state: The state from which to calculate state for the next step.
            u_dq: The action to apply to the environment.
            properties: Parameters and settings of the environment, that do not change over time.

        Returns:
            state: The computed state after the one step simulation.
        """
        system_state = state.physical_state
        omega_el = system_state.omega_el
        i_d = system_state.i_d
        i_q = system_state.i_q
        eps = system_state.epsilon

        args = (u_dq, properties.static_params)
        if properties.saturated:

            def vector_field(t, y, args):
                i_d, i_q = y
                u_dq, _ = args

                J_k = jnp.array([[0, -1], [1, 0]])
                i_dq = jnp.array([i_d, i_q])
                p_d = {q: interp(jnp.array([i_d, i_q])) for q, interp in self.motor_env.LUT_interpolators.items()}
                i_dq_norm = i_dq / properties.physical_constraints.i_d

                p_d["Psi_d"] = self.Psi_d(i_dq_norm)
                p_d["Psi_q"] = self.Psi_q(i_dq_norm)
                p_d["L_dd"] = self.L_dd(i_dq_norm)
                p_d["L_dq"] = self.L_dq(i_dq_norm)
                p_d["L_qd"] = self.L_qd(i_dq_norm)
                p_d["L_qq"] = self.L_qq(i_dq_norm)

                L_diff = jnp.column_stack([p_d[q] for q in ["L_dd", "L_dq", "L_qd", "L_qq"]]).reshape(2, 2)
                L_diff_inv = jnp.linalg.inv(L_diff)
                psi_dq = jnp.column_stack([p_d[psi] for psi in ["Psi_d", "Psi_q"]]).reshape(-1)
                di_dq_1 = jnp.einsum(
                    "ij,j->i",
                    (-L_diff_inv * properties.static_params.r_s),
                    i_dq,
                )
                di_dq_2 = jnp.einsum("ik,k->i", L_diff_inv, u_dq)
                di_dq_3 = jnp.einsum("ij,jk,k->i", -L_diff_inv, J_k, psi_dq) * omega_el
                i_dq_diff = di_dq_1 + di_dq_2 + di_dq_3
                d_y = i_dq_diff[0], i_dq_diff[1]

                return d_y

        else:

            def vector_field(t, y, args):
                i_d, i_q = y
                u_dq, params = args
                u_d = u_dq[0]
                u_q = u_dq[1]
                l_d = params.l_d
                l_q = params.l_q
                psi_p = params.psi_p
                r_s = params.r_s
                i_d_diff = (u_d + omega_el * l_q * i_q - r_s * i_d) / l_d
                i_q_diff = (u_q - omega_el * (l_d * i_d + psi_p) - r_s * i_q) / l_q
                d_y = i_d_diff, i_q_diff
                return d_y

        term = diffrax.ODETerm(vector_field)
        t0 = 0
        t1 = self.motor_env.tau
        y0 = tuple([i_d, i_q])
        env_state = self.motor_env._solver.init(term, t0, t1, y0, args)
        y, _, _, env_state, _ = self.motor_env._solver.step(term, t0, t1, y0, args, env_state, made_jump=False)

        i_d_k1 = y[0]
        i_q_k1 = y[1]

        if properties.saturated:
            torque = jnp.array(
                [self.motor_env.currents_to_torque_saturated(i_d=i_d_k1, i_q=i_q_k1, env_properties=properties)]
            )[0]
        else:
            torque = jnp.array([self.motor_env.currents_to_torque(i_d_k1, i_q_k1, properties)])[0]

        with jdc.copy_and_mutate(system_state, validate=False) as system_state_next:
            system_state_next.epsilon = step_eps(eps, omega_el, self.motor_env.tau, 1.0)
            system_state_next.i_d = i_d_k1
            system_state_next.i_q = i_q_k1
            system_state_next.torque = torque  # [0]

        with jdc.copy_and_mutate(state, validate=False) as state_next:
            state_next.physical_state = system_state_next
        return state_next

    #@partial(jax.jit, static_argnums=[0, 3])
    def step_expert(self, state, action, env_properties):
        """Computes state by simulating one step taking the deadtime into account.

        Args:
            system_state: The state from which to calculate state for the next step.
            action: The action to apply to the environment.
            properties: Parameters and settings of the environment, that do not change over time.

        Returns:
            state: The computed state after the one step simulation.
        """

        action = self.motor_env.constraint_denormalization(action, state, env_properties)

        action_buffer = jnp.array([state.physical_state.u_d_buffer, state.physical_state.u_q_buffer])

        if env_properties.static_params.deadtime > 0:

            updated_buffer = jnp.array([action[0], action[1]])
            u_dq = action_buffer
        else:
            updated_buffer = action_buffer

            u_dq = action

        next_state = self.ode_step(state, u_dq, env_properties)
        with jdc.copy_and_mutate(next_state, validate=True) as next_state_update:
            next_state_update.physical_state.u_d_buffer = updated_buffer[0]
            next_state_update.physical_state.u_q_buffer = updated_buffer[1]

        observation = self.motor_env.generate_observation(next_state_update, env_properties)
        return observation, next_state_update

    def Psi_d(self, i_dq_norm):
        return self.psi_dq_mlp(i_dq_norm)[0]  #  self.motor_env.LUT_interpolators["Psi_d"](i_dq)[0]

    def Psi_q(self, i_dq_norm):
        return self.psi_dq_mlp(i_dq_norm)[1]  #   self.motor_env.LUT_interpolators["Psi_q"](i_dq)[0]

    def Psi_d_physical(self, i_dq):
        i_dq_norm = i_dq / self.motor_env.env_properties.physical_constraints.i_d
        return self.Psi_d(i_dq_norm)

    def Psi_q_physical(self, i_dq):
        i_dq_norm = i_dq / self.motor_env.env_properties.physical_constraints.i_d
        return self.Psi_q(i_dq_norm)

    def l_d_dq(self, i_dq):
        return jax.grad(self.Psi_d_physical)(i_dq)

    def l_q_dq(self, i_dq):
        return jax.grad(self.Psi_q_physical)(i_dq)

    def L_dd(self, i_dq_norm):
        i_dq = i_dq_norm * self.motor_env.env_properties.physical_constraints.i_d
        return self.l_d_dq(i_dq)[0]

    def L_dq(self, i_dq_norm):
        i_dq = i_dq_norm * self.motor_env.env_properties.physical_constraints.i_d
        return self.l_d_dq(i_dq)[1]

    def L_qd(self, i_dq_norm):
        i_dq = i_dq_norm * self.motor_env.env_properties.physical_constraints.i_d
        return self.l_q_dq(i_dq)[0]

    def L_qq(self, i_dq_norm):
        i_dq = i_dq_norm * self.motor_env.env_properties.physical_constraints.i_d
        return self.l_q_dq(i_dq)[1]

    def L_matrix(self, i_dq):
        L_dd = self.L_dd(i_dq)
        L_dq = self.L_dq(i_dq)
        L_qd = self.L_qd(i_dq)
        L_qq = self.L_qq(i_dq)
        mat = jnp.array([[L_dd, L_dq], [L_qd, L_qq]])
        return mat


In [5]:
motor_env = PMSM(
    saturated=True,
    LUT_motor_name="BRUSA",
    batch_size=1,
    control_state=[], 
    static_params = {
            "p": 3,
            "r_s": 15e-3,
            "l_d": 0.37e-3,
            "l_q": 1.2e-3,
            "psi_p": 65.6e-3,
            "deadtime": 0,
    })

## Prediction Step Comparison

In [6]:
def build_grid(dim, low, high, points_per_dim):
    """Build a uniform grid of points in the given dimension."""
    xs = [jnp.linspace(low, high, points_per_dim) for _ in range(dim)]

    x_g = jnp.meshgrid(*xs)
    x_g = jnp.stack([_x for _x in x_g], axis=-1)
    x_g = x_g.reshape(-1, dim)

    assert x_g.shape[0] == points_per_dim**dim
    return x_g


def get_valid_points(data_grid, constr_func):
    valid_grid_point = jax.vmap(constr_func, in_axes=0)(data_grid) == 0
    constraint_data_points = data_grid[jnp.where(valid_grid_point == True)]
    return constraint_data_points


def valid_space_grid(constraint_function, data_dim, points_per_dim, min, max):
    hypercube_grid = build_grid(data_dim, min, max, points_per_dim)
    return get_valid_points(hypercube_grid, constraint_function)

In [7]:
class PredictionComparison(eqx.Module):
    grid: jax.Array
    action_dim: float
    obs_dim: float

    def __init__(self, grid, action_dim, obs_dim):
        assert grid.shape[-1] == (
            action_dim + obs_dim
        ), f"The grid dimension does not fit the action_dim and obs_dim. Grid dimension should be action_dim+obs_dim, but {grid.shape[-1]} and {action_dim}+{obs_dim} are given"
        self.grid = grid
        self.action_dim = action_dim
        self.obs_dim = obs_dim

    def __call__(self, model, model_gt, tau):
        pred = jax.vmap(model.step, in_axes=(0, 0,None))(self.grid[:, : self.obs_dim], self.grid[:, self.obs_dim :],tau)
        pred_gt = jax.vmap(model_gt.step, in_axes=(0, 0,None))(
            self.grid[:, : self.obs_dim], self.grid[:, self.obs_dim :], tau
        )
        return (pred[:,:2] - pred_gt[:,:2]), jnp.mean((pred[:,:2] - pred_gt[:,:2]) ** 2)

In [8]:
def constr_function(data_point):
    constr1=jax.nn.relu(((data_point[0]*0.5-0.5)**2+data_point[1]**2)-1)
    return constr1

In [9]:
grid=valid_space_grid(constr_function, 5, 16, -1, 1)
grid=jnp.concatenate([grid[:,:2],jnp.cos(jnp.pi*grid[:,3])[:,None],jnp.sin(jnp.pi*grid[:,3])[:,None],grid[:,-2:]],axis=1)
comp=PredictionComparison(grid=grid,action_dim=2,obs_dim=4)

In [10]:
class GtWrapper(eqx.Module):
    motor_env: PMSM = eqx.field(static=True)
    def __init__(self, motor_env):
        self.motor_env = motor_env

    def __call__(self, init_obs, actions, tau):

        def body_fun(carry, action):
            obs = carry
            obs = self.step(obs, action, tau)
            return obs, obs

        _, observations = jax.lax.scan(body_fun, init_obs, actions)
        observations = jnp.concatenate([init_obs[None, :], observations], axis=0)
        return observations

    def step(self, obs, action, tau):
        obs1, _ = self.motor_env.reset(self.motor_env.env_properties)  #
        obs1 = obs1.at[2].set((3 * 1500 / 60 * 2 * jnp.pi) / (2 * jnp.pi * 3 * 11000 / 60))
        obs1 = obs1.at[0].set(obs[0])
        obs1 = obs1.at[1].set(obs[1])
        obs1 = obs1.at[4].set(obs[2])
        obs1 = obs1.at[5].set(obs[3])
        state = self.motor_env.generate_state_from_observation(obs1, self.motor_env.env_properties)
        # obs,_= self.motor_env.step(state, action, self.motor_env.env_properties)
        obs, _ = self.motor_env.step(state, action, self.motor_env.env_properties)
        return jnp.concatenate([obs[0:2], obs[4:6]])

In [11]:
class BlackWrapperEps(eqx.Module):
    node: eqx.Module
    def __init__(self, node):
        self.node = node

    def __call__(self, init_obs, actions, tau):
        init_obs=init_obs[:2]
        return self.node(init_obs, actions, tau)

    def step(self, obs, action, tau):
        obs1 = obs[:2]
        return self.node.step(obs1, action, tau)

In [12]:
gt_wrap=GtWrapper(motor_env=motor_env)

In [13]:
jax_key = jax.random.PRNGKey(14)
node_struct= ExpertModel(motor_env=motor_env,psi_layer_sizes=[2,128,128,128,128,2],key=jax_key)
grey_node = eqx.tree_deserialise_leaves(f"final_models/grey_box/Model_4_128_2.eqx", node_struct) 

In [14]:
jax_key = jax.random.PRNGKey(2)
node_struct=NeuralEulerODE([6,128,128,128,128,4],key=jax_key)
black_node = eqx.tree_deserialise_leaves("final_models/black_box/Model_eps_4_128_9.eqx", node_struct)

In [15]:
jax_key = jax.random.PRNGKey(2)
node_struct=NeuralEulerODE([4,128,128,128,128,2],key=jax_key)
node_no_eps = eqx.tree_deserialise_leaves("final_models/black_box/Model_4_128_7.eqx", node_struct)
black_node_no_eps=BlackWrapperEps(node=node_no_eps)

In [16]:
def mse_metrics(size):
    values_grey=[]
    values_black=[]
    values_black_no_eps=[]
    for i in range(10):
        jax_key = jax.random.PRNGKey(2)
        node_struct=NeuralEulerODE(([4]+size+[2]),key=jax_key)
        node_no_eps = eqx.tree_deserialise_leaves(f"final_models/black_box/Model_{len(size)}_{size[0]}_{i+1}.eqx", node_struct)
        black_node_no_eps=BlackWrapperEps(node=node_no_eps)
        node_struct=NeuralEulerODE(([6]+size+[4]),key=jax_key)
        black_node = eqx.tree_deserialise_leaves(f"final_models/black_box/Model_eps_{len(size)}_{size[0]}_{i+1}.eqx", node_struct)
        node_struct= ExpertModel(motor_env=motor_env,psi_layer_sizes=([2]+size+[2]),key=jax_key)
        grey_node = eqx.tree_deserialise_leaves(f"final_models/grey_box/Model_{len(size)}_{size[0]}_{i+1}.eqx", node_struct)

        _,value_grey=comp(grey_node,gt_wrap,1e-4)
        _,value_black=comp(black_node,gt_wrap,1e-4)
        _,value_black_no_eps=comp(black_node_no_eps,gt_wrap,1e-4)
        values_grey.append(value_grey)
        values_black.append(value_black)
        values_black_no_eps.append(value_black_no_eps)
    
    return values_grey, values_black,  values_black_no_eps

### Different Network sizes

In [17]:
from tabulate import tabulate

In [18]:
mse_grey,mse_black,mse_black_no_eps=mse_metrics([128,128,128,128])
print([128,128,128,128])
print(tabulate([['Grey MSE', jnp.mean(jnp.array(mse_grey)),jnp.min(jnp.array(mse_grey)),jnp.max(jnp.array(mse_grey))], ['Black MSE',jnp.mean(jnp.array(mse_black)),jnp.min(jnp.array(mse_black)),jnp.max(jnp.array(mse_black))],['Black no eps MSE', jnp.mean(jnp.array(mse_black_no_eps)),jnp.min(jnp.array(mse_black_no_eps)),jnp.max(jnp.array(mse_black_no_eps))]], headers=['Network', 'Mean', "Best" ,"Worst"], tablefmt='orgtbl'))

[128, 128, 128, 128]
| Network          |        Mean |        Best |       Worst |
|------------------+-------------+-------------+-------------|
| Grey MSE         | 5.34204e-05 | 3.88253e-05 | 7.88713e-05 |
| Black MSE        | 0.000703244 | 0.000489531 | 0.000869475 |
| Black no eps MSE | 0.000593781 | 0.000498731 | 0.000801047 |


In [19]:
mse_grey,mse_black,mse_black_no_eps=mse_metrics([64,64,64])
print([64,64,64])
print(tabulate([['Grey MSE', jnp.mean(jnp.array(mse_grey)),jnp.min(jnp.array(mse_grey)),jnp.max(jnp.array(mse_grey))], ['Black MSE',jnp.mean(jnp.array(mse_black)),jnp.min(jnp.array(mse_black)),jnp.max(jnp.array(mse_black))],['Black no eps MSE', jnp.mean(jnp.array(mse_black_no_eps)),jnp.min(jnp.array(mse_black_no_eps)),jnp.max(jnp.array(mse_black_no_eps))]], headers=['Network', 'Mean', "Best" ,"Worst"], tablefmt='orgtbl'))

[64, 64, 64]
| Network          |        Mean |        Best |       Worst |
|------------------+-------------+-------------+-------------|
| Grey MSE         | 0.000157176 | 0.000103665 | 0.000259659 |
| Black MSE        | 0.00117136  | 0.00103674  | 0.00133074  |
| Black no eps MSE | 0.000962901 | 0.000730923 | 0.00116784  |


In [20]:
mse_grey,mse_black,mse_black_no_eps=mse_metrics([32,32])
print([32,32])
print(tabulate([['Grey MSE', jnp.mean(jnp.array(mse_grey)),jnp.min(jnp.array(mse_grey)),jnp.max(jnp.array(mse_grey))], ['Black MSE',jnp.mean(jnp.array(mse_black)),jnp.min(jnp.array(mse_black)),jnp.max(jnp.array(mse_black))],['Black no eps MSE', jnp.mean(jnp.array(mse_black_no_eps)),jnp.min(jnp.array(mse_black_no_eps)),jnp.max(jnp.array(mse_black_no_eps))]], headers=['Network', 'Mean', "Best" ,"Worst"], tablefmt='orgtbl'))

[32, 32]
| Network          |        Mean |        Best |        Worst |
|------------------+-------------+-------------+--------------|
| Grey MSE         | 10.972      | 0.000993369 | 105.118      |
| Black MSE        |  0.00293138 | 0.00194913  |   0.00613542 |
| Black no eps MSE |  0.00308058 | 0.00203764  |   0.00544367 |
